In [1]:
import sys, os, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go


In [2]:
sys.path.append(
    "/Users/wiegerscheurer/repos/physicspred"
)  # To enable importing from repository folders

In [3]:
from functions.physics import _rotate_90, _dir_to_vec, _vec_to_dir, _flip_dir
from functions.utilities import determine_sequence
from functions.analysis import get_precision, get_data, get_sensitivity, get_f1_score, get_rt, get_accuracy


#### Acquire data per sub

In [4]:
all_sub_names = ["melvin", "paulo", "yifan", "ann", "qifei", "mingyao", "bilge", "yanni"]
sub_stack = pd.DataFrame()

for sub_idx, sub in enumerate(all_sub_names):
    this_sub = get_data(subject=f"sub-{sub}", task="ball_hue")
    sub_stack = pd.concat([sub_stack, this_sub])
    
melvin = get_data(subject="sub-melvin", task = "ball_hue")
yanni = get_data(subject="sub-yanni", task = "ball_hue")
ann = get_data(subject="sub-ann", task = "ball_hue")



sub_stack.reset_index(inplace=True, drop=True)
sub_stack

,Unnamed: 0,trial,ball_speed,interactor,bounce,bounce_moment,random_bounce_direction,target_onset,speed_change,ball_change,...,start_pos,end_pos,abs_rfup,abs_rfright,abs_rfdown,abs_rfleft,sim_rfup,sim_rfright,sim_rfdown,sim_rfleft
0,0.0,1,7.5,45_top_r,False,NaN,NaN,4.075463,slower,True,...,left,right,"(0, 0)","(1, 1)","(0, 0)","(0, 0)","(0, 0)","(0, 1)","(1, 0)","(0, 0)"
1,1.0,2,7.5,45_top_r,True,3.870897,NaN,3.992594,slower,True,...,left,down,"(0, 0)","(1, 0)","(0, 1)","(0, 0)","(0, 0)","(0, 0)","(1, 1)","(0, 0)"
2,2.0,3,8.5,45_top_u,True,3.856596,NaN,NaN,NaN,False,...,down,left,"(1, 0)","(0, 0)","(0, 0)","(0, 1)","(0, 0)","(0, 0)","(0, 0)","(1, 1)"
3,3.0,4,8.5,none_u,True,NaN,left,NaN,NaN,False,...,down,left,"(1, 0)","(0, 0)","(0, 0)","(0, 1)","(1, 0)","(0, 0)","(0, 0)","(0, 1)"
4,4.0,5,8.0,45_top_r,False,NaN,NaN,4.021890,faster,True,...,left,right,"(0, 0)","(1, 1)","(0, 0)","(0, 0)","(0, 0)","(0, 1)","(1, 0)","(0, 0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,NaN,146,7.5,none_l,False,NaN,NaN,3.738320,slower,True,...,right,left,"(0, 0)","(0, 0)","(0, 0)","(1, 1)","(0, 0)","(0, 0)","(0, 0)","(1, 1)"
1196,NaN,147,8.0,135_bottom_d,True,3.631562,NaN,3.740338,faster,True,...,up,left,"(0, 0)","(0, 0)","(1, 0)","(0, 1)","(0, 0)","(0, 0)","(0, 0)","(1, 1)"
1197,NaN,148,7.5,135_top_u,False,NaN,NaN,NaN,NaN,False,...,down,up,"(1, 1)","(0, 0)","(0, 0)","(0, 0)","(0, 1)","(1, 0)","(0, 0)","(0, 0)"
1198,NaN,149,8.0,135_bottom_d,True,3.622973,NaN,3.728671,faster,True,...,up,left,"(0, 0)","(0, 0)","(1, 0)","(0, 1)","(0, 0)","(0, 0)","(0, 0)","(1, 1)"


In [12]:
yanni

,trial,ball_speed,interactor,bounce,bounce_moment,random_bounce_direction,target_onset,speed_change,ball_change,abs_congruent,...,start_pos,end_pos,abs_rfup,abs_rfright,abs_rfdown,abs_rfleft,sim_rfup,sim_rfright,sim_rfdown,sim_rfleft
0,1,8.0,none_d,False,NaN,NaN,3.833304,faster,True,True,...,up,down,"(0, 0)","(0, 0)","(1, 1)","(0, 0)","(0, 0)","(0, 0)","(1, 1)","(0, 0)"
1,2,8.5,45_bottom_d,False,NaN,NaN,NaN,NaN,False,True,...,up,down,"(0, 0)","(0, 0)","(1, 1)","(0, 0)","(0, 0)","(1, 0)","(0, 1)","(0, 0)"
2,3,7.5,135_bottom_d,False,NaN,NaN,NaN,NaN,False,True,...,up,down,"(0, 0)","(0, 0)","(1, 1)","(0, 0)","(0, 0)","(0, 0)","(0, 1)","(1, 0)"
3,4,8.0,none_l,False,NaN,NaN,NaN,NaN,False,True,...,right,left,"(0, 0)","(0, 0)","(0, 0)","(1, 1)","(0, 0)","(0, 0)","(0, 0)","(1, 1)"
4,5,8.0,45_top_u,True,3.641187,NaN,NaN,NaN,False,False,...,down,left,"(1, 0)","(0, 0)","(0, 0)","(0, 1)","(0, 0)","(0, 0)","(0, 0)","(1, 1)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,146,7.5,none_l,False,NaN,NaN,3.738320,slower,True,True,...,right,left,"(0, 0)","(0, 0)","(0, 0)","(1, 1)","(0, 0)","(0, 0)","(0, 0)","(1, 1)"
146,147,8.0,135_bottom_d,True,3.631562,NaN,3.740338,faster,True,False,...,up,left,"(0, 0)","(0, 0)","(1, 0)","(0, 1)","(0, 0)","(0, 0)","(0, 0)","(1, 1)"
147,148,7.5,135_top_u,False,NaN,NaN,NaN,NaN,False,True,...,down,up,"(1, 1)","(0, 0)","(0, 0)","(0, 0)","(0, 1)","(1, 0)","(0, 0)","(0, 0)"
148,149,8.0,135_bottom_d,True,3.622973,NaN,3.728671,faster,True,False,...,up,left,"(0, 0)","(0, 0)","(1, 0)","(0, 1)","(0, 0)","(0, 0)","(0, 0)","(1, 1)"


In [5]:
yan_rt = get_rt(yanni, sim_con = False, expol_con = False, return_df = True)
yan_rt["rt"]

27          NaN
57     0.692383
67          NaN
72          NaN
91          NaN
96     0.696251
104    0.717886
105    0.694027
112    0.755391
113    0.629673
131         NaN
139         NaN
143    1.004873
Name: rt, dtype: float64

In [36]:
this_sub = random.choice(all_sub_names)
print(this_sub)
sub = get_data(subject=f"sub-{this_sub}", task="ball_hue")

simc_xpolc = get_rt(df=sub, sim_con=True, expol_con=True, return_df=True)["rt"]
simc_xpoli = get_rt(df=sub, sim_con=True, expol_con=False, return_df=True)["rt"]
simi_xpolc = get_rt(df=sub, sim_con=False, expol_con=True, return_df=True)["rt"]
simi_xpoli = get_rt(df=sub, sim_con=False, expol_con=False, return_df=True)["rt"]


all_kinds = [simc_xpolc, simc_xpoli, simi_xpolc, simi_xpoli]

for kind in all_kinds:
    print(len(kind))
    print(kind.mean(), kind.std())





melvin
13
0.969199981064197 0.17684304408080773
29
0.9583410150045529 0.19854410886412746
25
1.0020756344681352 0.16864267889464926
7
nan nan


In [40]:
simi_xpoli = get_rt(df=get_data(subject=f"sub-melvin", task="ball_hue"), sim_con=False, expol_con=False, return_df=True)["rt"]
simi_xpoli

13    NaN
87    NaN
89    NaN
94    NaN
95    NaN
97    NaN
125   NaN
Name: rt, dtype: float64

In [63]:
trial_options = ["45_top_r", "45_top_u",
                 "45_bottom_l", "45_bottom_d",
                 "135_top_l", "135_top_u",
                 "135_bottom_r", "135_bottom_d"] + 2 * [f"none_{edge}" for edge in ["l", "r", "u", "d"]]

trial_options

['45_top_r',
 '45_top_u',
 '45_bottom_l',
 '45_bottom_d',
 '135_top_l',
 '135_top_u',
 '135_bottom_r',
 '135_bottom_d',
 'none_l',
 'none_r',
 'none_u',
 'none_d',
 'none_l',
 'none_r',
 'none_u',
 'none_d']

In [106]:
# Create randomised list of strings with "left" or "right" in random order


TypeError: arange() not supported for inputs with DType <class 'numpy.dtypes.StrDType'>.

In [207]:
n_trials = 16
rand_bounce_direction_options = ["left", "right"] * 2
random.shuffle(rand_bounce_direction_options)
print(rand_bounce_direction_options)

trials = determine_sequence(n_trials, trial_options, randomised=True)
# rand_bounce_directions = determine_sequence(
#     n_trials, rand_bounce_direction_options, randomised=True
# )

# trials == "none_l"
# Get the indices of trials where it is none_l
none_l_idx = [idx for idx, trial in enumerate(trials) if trial == "none_l"]


# Get boolean list of trials where it is none_l
none_l_bool = [trial == "none_l" for trial in trials]
# trial_array = None
# trial_array = np.array(trials) if trial_array is None else trial_array
# trial_array[none_l_bool] = "SOEP"
# list(trial_array)
# Make empty nan array of same size as none_l bool
trial_array = np.empty(len(trials), dtype=object)


for dir_idx, none_dir in enumerate(["l", "r", "u", "d"]):
    none_dir_bool = [trial == f"none_{none_dir}" for trial in trials]
    trial_array[none_dir_bool] = rand_bounce_direction_options[dir_idx]
    print(list(trial_array))



['right', 'left', 'right', 'left']
[None, None, 'right', None, 'right', None, None, None, None, None, None, None, None, None, None, None]
[None, None, 'right', 'left', 'right', None, None, 'left', None, None, None, None, None, None, None, None]
[None, None, 'right', 'left', 'right', None, None, 'left', None, None, None, None, 'right', None, 'right', None]
['left', None, 'right', 'left', 'right', None, None, 'left', None, None, 'left', None, 'right', None, 'right', None]


In [61]:
sim_cons = [True, False]
expol_cons = [True, False]
randsub = random.choice(all_sub_names)
print(f"Now looking at : {randsub}")
for sim_con in sim_cons:
    
    for expol_con in expol_cons:
        
        print(f"sim_con: {sim_con}, expol_con: {expol_con}")
        dat = get_data(subject=f"sub-{randsub}", task="ball_hue")
        dat_filt = dat[(dat["sim_congruent"] == sim_con) & (dat["abs_congruent"] == expol_con)]
        print(len(dat_filt))

Now looking at : melvin
sim_con: True, expol_con: True
23
sim_con: True, expol_con: False
50
sim_con: False, expol_con: True
52
sim_con: False, expol_con: False
25


In [11]:

# Create an empty list to store all data frames
all_data = []

# Define types and their colors
types = ['Simcon + Xpolcon', 'Simcon + Xpolinc', 'Siminc + Xpolcon', 'Siminc + Xpolinc']

# Generate a colormap
cmap = plt.get_cmap('YlOrRd')  # You can choose a different colormap if you prefer
col_factor = 50
col_icept = 150
# Create the colors dictionary
type_colors = {name: f'rgba({int(cmap(i*col_factor + col_icept)[0]*255)}, {int(cmap(i*col_factor + col_icept)[1]*255)}, {int(cmap(i*col_factor + col_icept)[2]*255)}, 0.5)' 
          for i, name in enumerate(types)}

# Loop through all subjects (using the same structure as your original code)
for sub_name in all_sub_names:
    sub = get_data(subject=f"sub-{sub_name}", task="ball_hue")
    
    simc_xpolc = get_rt(df=sub, sim_con=True, expol_con=True, return_df=True)["rt"]
    simc_xpoli = get_rt(df=sub, sim_con=True, expol_con=False, return_df=True)["rt"]
    simi_xpolc = get_rt(df=sub, sim_con=False, expol_con=True, return_df=True)["rt"]
    simi_xpoli = get_rt(df=sub, sim_con=False, expol_con=False, return_df=True)["rt"]
    
    # Create DataFrames without subject column - we'll aggregate across all subjects
    simc_xpolc_df = pd.DataFrame({
        'Reaction Time': simc_xpolc,
        'Type': 'Simcon + Xpolcon'
    })
    
    simc_xpoli_df = pd.DataFrame({
        'Reaction Time': simc_xpoli,
        'Type': 'Simcon + Xpolinc'
    })
    
    simi_xpolc_df = pd.DataFrame({
        'Reaction Time': simi_xpolc,
        'Type': 'Siminc + Xpolcon'
    })
    
    simi_xpoli_df = pd.DataFrame({
        'Reaction Time': simi_xpoli,
        'Type': 'Siminc + Xpolinc'
    })
    
    # Append all DataFrames to our list
    all_data.extend([simc_xpolc_df, simc_xpoli_df, simi_xpolc_df, simi_xpoli_df])

# Combine all the data into a single DataFrame
combined_data = pd.concat(all_data, ignore_index=True)

# Calculate mean reaction time for each type
type_means = combined_data.groupby('Type')['Reaction Time'].mean().to_dict()

# Create the plot
fig = go.Figure()

# Add violin plots for each type
for i, type_name in enumerate(types):
    subset = combined_data[combined_data['Type'] == type_name]
    
    fig.add_trace(go.Violin(
        x=[type_name] * len(subset),  # Use type name directly as x value
        y=subset['Reaction Time'],
        name=type_name,
        legendgroup=type_name,
        showlegend=True,
        box_visible=True,
        meanline_visible=True,
        points='all',
        jitter=0.2,
        pointpos=0.5,
        line_color=type_colors[type_name],
        side='negative',
        width=.5,
        spanmode='soft'
    ))

# Connect the means with a line
fig.add_trace(go.Scatter(
    x=types,
    y=[type_means[t] for t in types],
    mode='lines+markers',
    line=dict(color='rgba(0, 0, 0, 0.7)', width=8),
    marker=dict(
        size=14,  # Increased marker size
        color=[type_colors[t].replace('0.5', '1') for t in types],
        line=dict(color='black', width=4)
    ),
    name='Mean RT',
    hovertemplate='Mean: %{y:.3f}s<extra></extra>'
))

# Define a common font style for consistent text appearance
font_style = dict(
    family="Arial, sans-serif",
    size=20,  # Larger font size
    color="black"
)

# Update layout with enhanced text styling
fig.update_layout(
    title=dict(
        text="Average Reaction Times Across All Subjects\n\n",
        font=dict(size=25, family="Arial, sans-serif", color="black"),
        x=0.5,
        y=.985
    ),
    yaxis=dict(
        title=dict(
            text="Reaction Time (s)",
            font=font_style
        ),
        tickfont=font_style,
        tickwidth=2,
        showline=True,
        linewidth=3,
        linecolor='black'
    ),
    xaxis=dict(
        title=dict(
            text="Response Type",
            font=font_style
        ),
        tickmode='array',
        tickvals=types,
        tickfont=font_style,
        tickwidth=2,
        showline=True,
        linewidth=3,
        linecolor='black'
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.01,
        xanchor="center",
        x=0.5,
        font=dict(size=18, family="Arial, sans-serif"),
        borderwidth=2,
        bordercolor="White",
        title=dict(
            text="Response Type",
            font=dict(size=18, family="Arial, sans-serif")
        )
    ),
    violinmode='overlay',
    # height=1200,
    # width=1200,
    height=800,
    width=800,
    font=font_style,  # Default font for all other text elements
    plot_bgcolor='white',
    paper_bgcolor='white'
)

# Add annotations for the mean values
for i, type_name in enumerate(types):
    fig.add_annotation(
        x=type_name,
        y=type_means[type_name],
        text=f"{type_means[type_name]:.3f}s",
        showarrow=True,
        arrowhead=2,
        arrowsize=1.5,  # Slightly larger arrowhead
        arrowwidth=3,    # Thicker arrow
        arrowcolor='black',
        ax=120,
        ay=-90,
        font=dict(size=25, color='black', family="Arial, sans-serif")
    )

# Make the axis lines thicker
fig.update_xaxes(mirror=True, ticks='outside', tickwidth=3, ticklen=10, showgrid=False)
fig.update_yaxes(mirror=True, ticks='outside', tickwidth=3, ticklen=10, showgrid=True, gridwidth=1, gridcolor='lightgray')

fig.show()




In [34]:

# Create an empty list to store all data frames
all_data = []

# Define types and their colors
types = ['Abstraction', 'Simulation', 'Both']

# Generate a colormap
cmap = plt.get_cmap('YlOrRd')  # You can choose a different colormap if you prefer
col_factor = 50
col_icept = 150
# Create the colors dictionary
type_colors = {name: f'rgba({int(cmap(i*col_factor + col_icept)[0]*255)}, {int(cmap(i*col_factor + col_icept)[1]*255)}, {int(cmap(i*col_factor + col_icept)[2]*255)}, 0.5)' 
          for i, name in enumerate(types)}

# Loop through all subjects (using the same structure as your original code)
for sub_name in all_sub_names:
    sub = get_data(subject=f"sub-{sub_name}", task="ball_hue")
    # Get your data as before
    rt_dubs = get_rt(sub, hypothesis="both", include_dubtrials="only", return_df=True)
    rt_separate = get_rt(sub, hypothesis="both", include_dubtrials=False, return_df=True)
    sim_rt = rt_separate["simulation"]["rt"]
    abs_rt = rt_separate["abstraction"]["rt"]
    both_rt = rt_dubs["sim + abs"]["rt"]
    
    # Create DataFrames without subject column - we'll aggregate across all subjects
    sim_df = pd.DataFrame({
        'Reaction Time': sim_rt,
        'Type': 'Simulation'
    })
    
    abs_df = pd.DataFrame({
        'Reaction Time': abs_rt,
        'Type': 'Abstraction'
    })
    
    both_df = pd.DataFrame({
        'Reaction Time': both_rt,
        'Type': 'Both'
    })
    
    # Append all DataFrames to our list
    all_data.extend([abs_df, sim_df, both_df])

# Combine all the data into a single DataFrame
combined_data = pd.concat(all_data, ignore_index=True)

# Calculate mean reaction time for each type
type_means = combined_data.groupby('Type')['Reaction Time'].mean().to_dict()

# Create the plot
fig = go.Figure()

# Add violin plots for each type
for i, type_name in enumerate(types):
    subset = combined_data[combined_data['Type'] == type_name]
    
    fig.add_trace(go.Violin(
        x=[type_name] * len(subset),  # Use type name directly as x value
        y=subset['Reaction Time'],
        name=type_name,
        legendgroup=type_name,
        showlegend=True,
        box_visible=True,
        meanline_visible=True,
        points='all',
        jitter=0.2,
        pointpos=0.5,
        line_color=type_colors[type_name],
        side='negative',
        width=.5,
        spanmode='soft'
    ))

# Connect the means with a line
fig.add_trace(go.Scatter(
    x=types,
    y=[type_means[t] for t in types],
    mode='lines+markers',
    line=dict(color='rgba(0, 0, 0, 0.7)', width=8),
    marker=dict(
        size=14,  # Increased marker size
        color=[type_colors[t].replace('0.5', '1') for t in types],
        line=dict(color='black', width=4)
    ),
    name='Mean RT',
    hovertemplate='Mean: %{y:.3f}s<extra></extra>'
))

# Define a common font style for consistent text appearance
font_style = dict(
    family="Arial, sans-serif",
    size=20,  # Larger font size
    color="black"
)

# Update layout with enhanced text styling
fig.update_layout(
    title=dict(
        text="Average Reaction Times Across All Subjects\n\n",
        font=dict(size=25, family="Arial, sans-serif", color="black"),
        x=0.5,
        y=.985
    ),
    yaxis=dict(
        title=dict(
            text="Reaction Time (s)",
            font=font_style
        ),
        tickfont=font_style,
        tickwidth=2,
        showline=True,
        linewidth=3,
        linecolor='black'
    ),
    xaxis=dict(
        title=dict(
            text="Response Type",
            font=font_style
        ),
        tickmode='array',
        tickvals=types,
        tickfont=font_style,
        tickwidth=2,
        showline=True,
        linewidth=3,
        linecolor='black'
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.01,
        xanchor="center",
        x=0.5,
        font=dict(size=18, family="Arial, sans-serif"),
        borderwidth=2,
        bordercolor="White",
        title=dict(
            text="Response Type",
            font=dict(size=18, family="Arial, sans-serif")
        )
    ),
    violinmode='overlay',
    # height=1200,
    # width=1200,
    height=800,
    width=800,
    font=font_style,  # Default font for all other text elements
    plot_bgcolor='white',
    paper_bgcolor='white'
)

# Add annotations for the mean values
for i, type_name in enumerate(types):
    fig.add_annotation(
        x=type_name,
        y=type_means[type_name],
        text=f"{type_means[type_name]:.3f}s",
        showarrow=True,
        arrowhead=2,
        arrowsize=1.5,  # Slightly larger arrowhead
        arrowwidth=3,    # Thicker arrow
        arrowcolor='black',
        ax=120,
        ay=-90,
        font=dict(size=25, color='black', family="Arial, sans-serif")
    )

# Make the axis lines thicker
fig.update_xaxes(mirror=True, ticks='outside', tickwidth=3, ticklen=10, showgrid=False)
fig.update_yaxes(mirror=True, ticks='outside', tickwidth=3, ticklen=10, showgrid=True, gridwidth=1, gridcolor='lightgray')

fig.show()